# TensorFlow - basics

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

import tensorflow as tf
print(f'Hello from TF version {tf.__version__}')

#### Note on TF versioning
Everything we show here today is based on TensorFlow 2+ (specifically, 2.3.0). TF 1 is significantly different and harder to use, so it is recommended that you stay away from it if possible. If you're already familiar with TF 1, you'll see that a lot of concepts have changed since TF 2 is (mostly) backwards incompatible. When searching on StackOverflow or similar, be careful!

### TF can be like NumPy.
The fundamental object in TensorFlow (TF) is the tensor. A `tf.Tensor` represents a multi-dimensional array with a uniform datatype (similarly to NumPy's `np.ndarray`).

In [ ]:
a = tf.constant(42)
b = tf.constant([8, -6])
c = tf.constant([[1, 2], [5, 3], [4, 6]])
d = tf.constant([[1, 0], [-1, 5]])
a + b * (c @ d)

Numpy-like functions are readily available, indexing & slicing and reshaping work like you would expect them to.

In [ ]:
print(f'Max of c: {tf.reduce_max(c)}')
print(f'Indices of column maxima of c: {tf.argmax(c)}')
print(f'Slice and reshape result: {tf.reshape(c[1:, :], [-1, ])}')

Numpy interoperability is usually not a problem. Most TF operations automatically convert a Numpy array (or even a Python list!) to a tensor. You can transform a `tf.Tensor` to a Numpy array by calling `.numpy()` on it.

In [ ]:
import numpy as np
d @ np.array([[3], [9]])

Note that TF also includes ragged (uneven) tensors, string tensors and sparse tensors, all of which can be useful.

In [ ]:
uneven_list = [['zero'], ['one', 'one'], ['two', 'three', 'five'], ['eight', 'thirteen']]
print(f'Ragged (uneven) tensor: \n{tf.ragged.constant(uneven_list)}\n')

sparse_tensor = tf.sparse.SparseTensor(indices=[[0, 0], [1, 2]],
                                       values=[1, 2],
                                       dense_shape=[3, 4])
print(f'Sparse tensor: \n{sparse_tensor}\n')
print(f'Sparse tensor in dense representation: \n{tf.sparse.to_dense(sparse_tensor)}')

### What can I do in TF that I can't do in Numpy?

When you think about TF, you usually think "deep learning". That's correct - it is most frequently used in DL. However, you can theoretically also do DL in Numpy. It won't be terribly efficient and you'll have to implement everything except matrix manipulation, but still, it can be done. If you add Scipy to the mix, you also get optimization procedures and numerical differentiation, which is better, but not optimal. In Tensorflow, you get analytic derivatives built-in. Each model you construct in TF is a graph of operations, where each operation has a forward and a backward pass defined - the backward pass being the gradient calculation. This automatic differentiation is key to implementing effective custom models!

### How do I use automatic differentiation?
We'll start with variables. In TF, tensors are immutable, which means that if you perform an operation on a tensor, a copy will be created and the original left unmodified. For something that will be updated frequently (say, model parameters during optimization), this won't do and we'll need to use a `tf.Variable`. A variable behaves similarly to a tensor, but it is mutable and can be modified.

In [ ]:
w = tf.Variable(0.1)
b = tf.Variable(-0.1)

It's easy to get the variables' gradients with respect to a function. Let's create some data, then use our weight and bias to get an estimate and compute the loss. We'll choose the line $y = x + 2 + U(-0.05, 0.05)$ as our target relationship to model and use the mean squared error as a loss function. The gradients will be computed with respect to MSE with the `tf.GradientTape` API.

In [ ]:
# Create uniform points on [-1, 1]
x = tf.cast(tf.linspace(-1, 1, 21), tf.float32)
# y = x + 2 + U(-0.05, 0.05)
y = x + tf.random.uniform(x.shape, -0.05, 0.05, seed=117) + 2

# The GradientTape "records" operations computed inside its context and stores gradients for all variables used in it.
with tf.GradientTape() as tape:
    y_hat = x * w + b
    mse = tf.reduce_mean((y - y_hat) ** 2)

# Derivatives of w and b with respect to MSE
dw, db = tape.gradient(mse, [w, b])
dw, db

Since TF readily offers us gradients, let's use them! We can easily use gradient descent to implement a simple linear regression example. We'll try to fit the line (i.e. minimize the MSE) by running 50 iterations of GD.

In [ ]:
# Learning rate and iterations
lr = 0.1
iters = 50

for _ in range(iters):
    with tf.GradientTape() as tape:
        y_hat = x * w + b
        mse = tf.reduce_mean(tf.pow(y - y_hat, 2))
    
    dw, db = tape.gradient(mse, [w, b])
    # GD update step
    w.assign(w - lr * dw)
    b.assign(b - lr * db)

# Error
print(f'Mean squared error: {mse:.4f}')
    
# Plot the fit
sns.relplot(data = pd.DataFrame({'x': x, 'y_true': y, 'y_hat': x * w + b}))

We see that we managed to achieve a good fit. Note that automatic differentiation works with (almost) all operations you can use in TF, which means you can build arbitrarily large and complex models and still have gradients for all parameters handy. 

However, this is still a bit awkward to use - we still need to implement the optimization procedure ourselves. What we've shown so far is actually a very low-level TensorFlow API and is not what you would usually use to create models unless you really need fine-grained control. In the next notebook, we'll move to the Keras high-level API, which is much more user-friendly.